# Tarea: La Subhasta del Peix

El siguiente código implementa una **simulación de subastas** modelada como un sistema multiagente en Python utilizando la biblioteca **osBrain**. En esta simulación, diferentes tipos de comerciantes (básicos, ricos y pobres) interactúan con un operador para comprar pescado en un mercado basado en la tradición catalana de **“la Subhasta del Peix”**. Esta subasta sigue el modelo de **subasta holandesa**, donde los precios de los productos disminuyen hasta que un comprador realiza una oferta o el precio alcanza un valor mínimo.

## **Introducción**

Hasta ahora, trabajamos con agentes que interactuaban dentro de un entorno o ignoraban a sus compañeros. Sin embargo, uno de los elementos clave en sistemas multiagente es la comunicación entre agentes. En esta tarea, modelaremos una subasta holandesa en Python, donde el operador y los comerciantes interactúan enviándose mensajes a través de canales de comunicación.

### **Objetivo de la Simulación**
- Modelar una subasta holandesa para la compra y venta de tres tipos de pescado: **merluza (H)**, **lenguado (S)** y **atún (T)**.
- Garantizar que los comerciantes gestionen su presupuesto para adquirir pescado, dando prioridad a sus preferencias.
- Explorar cómo los agentes interactúan a través del paso de mensajes para negociar y realizar compras.

---

## **Configuración General**

1. **Operador (Operator):**
   - Es el propietario de la subasta y administra la venta de los productos.
   - Cada pescado tiene un **precio inicial** y un **precio mínimo** (valor de descarte).
   - Publica información de cada producto y procesa las ofertas de los comerciantes.

2. **Comerciantes (Merchants):**
   - Representan a los posibles compradores.
   - Cada comerciante tiene un **presupuesto** inicial y una **preferencia** por un tipo de pescado.
   - El objetivo es adquirir al menos un pescado de cada tipo y priorizar la compra del tipo preferido.

---

## **Implementación**

### **Dinámica de la Subasta**
- **Rondas de Subasta:** En cada ronda, el operador publica un nuevo pescado con un precio inicial.
- **Reducción de Precios:** Si ningún comerciante puja, el precio disminuye en intervalos hasta alcanzar el precio mínimo.
- **Venta:** Si un comerciante realiza una puja, el pescado se vende al precio actual y la subasta pasa al siguiente producto.
- **Descarte:** Si el precio mínimo se alcanza sin que haya pujas, el pescado se descarta.

### **Participantes**
- **Operador:**
  - Publica cada pescado y reduce el precio gradualmente si no hay pujas.
  - Procesa las pujas y confirma la venta.
- **Comerciantes:**
  - Evalúan los productos en función de su precio, tipo y presupuesto.
  - Realizan pujas estratégicas según su preferencia y necesidad de completar su inventario.

---

## **Escenario Configurado**

### **Operador**
- Tipo: Finito con evaluación de calidad.
- Número máximo de pescados a subastar: 5.

### **Comerciantes**
1. **Básicos:**
   - Presupuesto inicial: 100.
   - Lógica estándar para realizar pujas.
   - Número configurado: 3.

2. **Ricos:**
   - Presupuesto inicial: 500.
   - Prioridad en pescados preferidos a precios altos.
   - Número configurado: 0.

3. **Pobres:**
   - Presupuesto inicial: 50.
   - Solo compran pescados con descuentos significativos.
   - Número configurado: 2.

---

## **Propósito del Sistema Multiagente**
Esta simulación permite analizar:
- La interacción entre agentes mediante el paso de mensajes.
- La dinámica del mercado en un entorno de subasta holandesa.
- Cómo las preferencias y presupuestos afectan las decisiones de compra de los comerciantes.

In [1]:
# Cell 1: Configuration
# ==========================
# Auction Simulation Config
# ==========================

# Select the type of operator for the simulation:
# 1 - Infinite Operator (no quality)
# 2 - Finite Operator (no quality)
# 3 - Infinite Operator with Quality
# 4 - Finite Operator with Quality
# Default: 1
operator_type: 4

# For finite operators (type 2 and 4 only):
# Specify the total number of fish to sell before the simulation stops.
# Example: 10
# Default: 10
total_fish_to_sell: 5

# Set the number of Basic Merchants:
# Basic merchants have a budget of 100 and standard logic.
# Example: 3
# Default: 0
num_basic_merchants: 3

# Set the number of Rich Merchants:
# Rich merchants have a higher budget of 500 and focus on preferred fish.
# Example: 2
# Default: 0
num_rich_merchants: 0

# Set the number of Poor Merchants:
# Poor merchants have a budget of 50 and only buy heavily discounted fish.
# Example: 1
# Default: 0
num_poor_merchants: 2

# ==========================
# End of Config
# ==========================

# ==========================
# Auction Simulation Config
# ==========================

# Select the type of operator for the simulation:
# 1 - Infinite Operator (no quality)
# 2 - Finite Operator (no quality)
# 3 - Infinite Operator with Quality
# 4 - Finite Operator with Quality
# Default: 1
operator_type: 4

# For finite operators (type 2 and 4 only):
# Specify the total number of fish to sell before the simulation stops.
# Example: 10
# Default: 10
total_fish_to_sell: 5

# Set the number of Basic Merchants:
# Basic merchants have a budget of 100 and standard logic.
# Example: 3
# Default: 0
num_basic_merchants: 3

# Set the number of Rich Merchants:
# Rich merchants have a higher budget of 500 and focus on preferred fish.
# Example: 2
# Default: 0
num_rich_merchants: 0

# Set the number of Poor Merchants:
# Poor merchants have a budget of 50 and only buy heavily discounted fish.
# Example: 1
# Default: 0
num_poor_merchants: 2

# ==========================
# End of Config
# ==========================


### Agentes `Comerciantes`

El siguiente código define tres tipos de **agentes comerciantes** que participan en una **subasta de pescado**. 
Los agentes heredan de una clase base Merchant y tienen comportamientos personalizados para decidir si compran pescado según su presupuesto, preferencias y estrategias específicas.

---

### **Clase: Merchant**
- **Propiedades principales:**
  - **Inventario**: Los productos adquiridos se almacenan aquí.
  - **Presupuesto inicial**: Cada comerciante comienza con un presupuesto que puede ajustarse en las subclases.
  - **Preferencia**: Cada comerciante tiene un tipo de pescado preferido (H, S o T), elegido aleatoriamente.
  - **Umbrales de precio**: Determina cuánto están dispuestos a pagar según la calidad (buena, normal, mala).

- **Métodos principales:**
  - **`on_product_info`**: Decide si pujar o no por un producto, considerando:
    - El tipo y calidad del pescado.
    - El precio en relación con los umbrales de preferencia.
    - Las condiciones del inventario.
  - **`on_confirmation`**: Maneja la confirmación de una compra, actualizando:
    - El presupuesto.
    - El inventario y los umbrales de precio.
  - **`on_operator_message`**: Procesa mensajes del operador sobre las subastas.

---

### **Subclases de Comerciantes**

#### **1. BasicMerchant**
- **Características:**
  - Presupuesto inicial: **100**.
  - Lógica estándar para decidir compras basada en preferencias y precios.

#### **2. RichMerchant**
- **Características:**
  - Presupuesto inicial: **500**.
  - Siempre compra pescado preferido si el precio es aceptable.
  - Umbral de precio fijo (30) para pescado de calidad preferida (sin disminución del umbral tras compras).

#### **3. PoorMerchant**
- **Características:**
  - Presupuesto inicial: **50**.
  - Solo compra pescado cuando el precio está muy descontado (<=15).
  - Evita cualquier gasto significativo en pescado no prioritario.

---

### **Flujo General**
1. **Inicio**: Los comerciantes configuran su presupuesto, preferencias y lógica de compra.
2. **Recepción de información del producto**: Cada comerciante evalúa el pescado en subasta (precio, calidad, preferencia).
3. **Decisión de puja**: Dependiendo de las reglas personalizadas de cada tipo de comerciante, decide si pujar.
4. **Confirmación de compra**: Se actualiza el inventario y el presupuesto si se confirma la compra.

In [7]:
import random
from osbrain import Agent

class Merchant(Agent):
    def on_init(self):
        self.inventory = {}
        self.budget = 100  # Default budget, adjustable by subclasses
        self.preference = random.choice(['H', 'S', 'T'])  # Random fish type preference
        self.log_info(f"My preference is: {self.preference}")
        self.fish_types = ['H', 'S', 'T']
        self.current_auctions = {}

        # Inventory counts per fish type
        self.inventory_counts = {fish_type: 0 for fish_type in self.fish_types}

        # Quality-based price thresholds and minimums
        self.preferred_price_thresholds = {'good': 30, 'normal': 20, 'bad': 10}
        self.preferred_price_minimums = {'good': 10, 'normal': 10, 'bad': 10}

    def get_name(self):
        """
        Return the name of the merchant for external access.
        """
        return self.name

    def on_operator_message(self, message):
        """Handles incoming messages from the operator."""
        message_type = message.get('message_type')
        if message_type == 'auction_info':
            self.on_product_info(message)
        elif message_type == 'confirmation':
            self.on_confirmation(message)

    def on_product_info(self, message):
        """
        Handles product auction information and decides whether to bid.
        Supports quality as an optional attribute.
        """
        product_number = message.get('product_number')
        product_type = message.get('product_type')
        price = message.get('price')
        quality = message.get('quality', None)  # Defaults to None if not provided

        # Skip if auction is closed or budget is insufficient
        if self.budget < price or self.current_auctions.get(product_number, {}).get('status') == 'closed':
            return

        # Store auction details
        self.current_auctions[product_number] = {
            'product_type': product_type,
            'quality': quality,
            'price': price,
            'status': 'open'
        }

        # Determine the threshold for quality (default to mid-range if not specified)
        threshold = self.preferred_price_thresholds.get(quality, 20)

        # Buying logic
        should_buy = False
        if product_type == self.preference:
            # Buy preferred fish within acceptable price range
            if price <= threshold:
                should_buy = True
        else:
            # Buy non-preferred fish if discounted and inventory is empty
            if self.inventory_counts[product_type] == 0 and price <= (threshold / 2):
                should_buy = True

        if should_buy:
            self.log_info(f"Attempting to buy Fish {product_number} at price {price} with quality {quality}")
            bid = {
                'merchant_id': self.name,
                'product_number': product_number,
            }
            # Send bid to operator
            self.send('bid_channel', bid)
            # Mark auction as pending
            self.current_auctions[product_number]['status'] = 'pending'

    def on_confirmation(self, message):
        """
        Handles confirmation of purchase and updates inventory, budget, and price thresholds.
        """
        merchant_id = message.get('merchant_id')
        if merchant_id != self.name:
            return  # Ignore confirmations not meant for this merchant

        product_number = message.get('product_number')
        price = message.get('price')
        product_type = message.get('product_type')
        quality = message.get('quality', None)

        self.log_info(f"Purchase confirmed for Fish {product_number} at price {price} with quality {quality}")
        self.budget -= price

        # Update inventory
        self.inventory[product_number] = {
            'type': product_type,
            'quality': quality,
            'price': price
        }
        self.inventory_counts[product_type] += 1
        self.log_info(f"Remaining budget: {self.budget}")

        # Mark auction as closed
        self.current_auctions[product_number]['status'] = 'closed'

        # Adjust thresholds if preferred fish is bought
        if product_type == self.preference and quality in self.preferred_price_thresholds:
            old_threshold = self.preferred_price_thresholds[quality]
            self.preferred_price_thresholds[quality] *= 0.8  # Reduce threshold by 20%
            if self.preferred_price_thresholds[quality] < self.preferred_price_minimums[quality]:
                self.preferred_price_thresholds[quality] = self.preferred_price_minimums[quality]
            self.log_info(
                f"Threshold for {quality} quality reduced from {old_threshold:.2f} to {self.preferred_price_thresholds[quality]:.2f}"
            )

    def on_exit(self):
        """Optional cleanup logic."""
        self.log_info("Merchant shutting down.")



class BasicMerchant(Merchant):
    def on_init(self):
        super().on_init()
        self.budget = 100


class RichMerchant(Merchant):
    def on_init(self):
        super().on_init()
        self.budget = 500
        # Rich merchants always accept the max price for preferred fish
        self.preferred_price_threshold = 30
        self.preferred_price_minimum = 30  # No decrease

    def on_confirmation(self, message):
        """
        Handles confirmation of purchase and updates inventory, budget, and price thresholds.
        """
        merchant_id = message.get('merchant_id')
        if merchant_id != self.name:
            return  # Ignore confirmations not meant for this merchant

        product_number = message.get('product_number')
        price = message.get('price')
        product_type = message.get('product_type')
        quality = message.get('quality', 'N/A')  # Ensure quality is logged correctly

        self.log_info(f"Purchase confirmed for Fish {product_number} at price {price} with quality {quality}")
        self.budget -= price

        # Update inventory with quality
        self.inventory[product_number] = {
            'type': product_type,
            'quality': quality,
            'price': price
        }
        self.inventory_counts[product_type] += 1
        self.log_info(f"Remaining budget: {self.budget}")


class PoorMerchant(Merchant):
    def on_init(self):
        super().on_init()
        self.budget = 50
        # Set a low preferred price threshold
        self.preferred_price_threshold = 15
        self.preferred_price_minimum = 10

    def on_product_info(self, message):
        # Override buying logic to only buy at heavy discounts
        product_number = message.get('product_number')
        product_type = message.get('product_type')
        price = message.get('price')

        if self.budget >= price and self.current_auctions.get(product_number, {}).get('status') != 'closed':
            self.current_auctions[product_number] = {
                'product_type': product_type,
                'price': price,
                'status': 'open'
            }
            should_buy = False

            if price <= 15:
                # Only buy if price is heavily discounted
                should_buy = True

            if should_buy:
                self.log_info(f"Attempting to buy Fish {product_number} at price {price}")
                bid = {
                    'merchant_id': self.name,
                    'product_number': product_number,
                }
                self.send('bid_channel', bid)
                self.current_auctions[product_number]['status'] = 'pending'

### Agente `Operador`

El operador es el **propietario de la subasta**, responsable de gestionar y publicar los productos. Cada producto tiene un **precio inicial** y un **precio mínimo**. Solo habrá un operador en la simulación.

---

### **Clase Base: `Operator`**
1. **Propósito:** Gestiona el flujo de la subasta y la interacción con los comerciantes.
2. **Atributos Principales:**
   - **`publish_channel` y `bid_channel`:** Canales para enviar información sobre los productos y recibir pujas de los comerciantes.
   - **`fish_types`:** Tipos de pescado disponibles (H, S, T).
   - **`current_auction`:** Producto actualmente en subasta con su precio inicial y mínimo.
   - **`transactions`:** Registro de todas las transacciones realizadas.
   - **`running`:** Bandera que indica si la subasta está activa.
3. **Métodos Importantes:**
   - **`start_auction`:** Inicia la subasta.
   - **`auction_next_fish`:** Prepara y publica el siguiente producto en subasta (implementado en subclases).
   - **`send_fish_info`:** Publica los detalles del producto en subasta (precio inicial, tipo).
   - **`on_bid`:** Procesa pujas recibidas, vendiendo el producto si la puja es válida.
   - **`check_for_replies`:** Reduce el precio del producto si no se reciben pujas (definido en subclases).

---

### **Clases Derivadas:**
1. **`OperatorInfinite`:** 
   - Maneja subastas con un inventario "infinito" de productos. 
   - Publica cada producto con un precio inicial y reduce el precio hasta alcanzar el valor mínimo si no hay compradores.

2. **`OperatorFinite`:** 
   - Maneja subastas con un número limitado de productos para vender.
   - Termina la subasta tras alcanzar el número configurado de productos vendidos.

3. **Clases con Calidad (`Quality`):**
   - **`OperatorInfiniteQuality` y `OperatorFiniteQuality`:** Extienden la funcionalidad para incluir la calidad (buena, normal, mala) en cada producto, lo que influye en la decisión de compra de los comerciantes.

---

### **Flujo General del `Operator`:**
1. **Publicación del Producto:**
   - Cada producto tiene un precio inicial y un precio mínimo.
   - La información del producto se publica para que los comerciantes puedan pujar.
2. **Procesamiento de Pujas:**
   - Si un comerciante realiza una puja válida, el producto se vende al precio actual.
   - Se registra la transacción y se publica una confirmación.
3. **Reducción de Precio:**
   - Si no hay pujas, el precio se reduce de forma iterativa hasta alcanzar el precio mínimo.
   - Si el precio mínimo es alcanzado y no hay compradores, el producto se marca como "no vendido".
4. **Finalización:**
   - La subasta termina cuando se cumplen las condiciones (inventario agotado o límite de ventas alcanzado).

import random
from osbrain import Agent

class Operator(Agent):
    def on_init(self):
        # PUB socket to broadcast auction info and confirmations
        self.publish_address = self.bind('PUB', alias='publish_channel')
        # PULL socket to receive bids from merchants
        self.bid_address = self.bind('PULL', alias='bid_channel', handler=self.on_bid)
        self.fish_types = ['H', 'S', 'T']
        self.fish_index = 0
        self.transactions = []
        self.current_auction = None
        self.running = True  # Indicates whether the auction is running


    def start_auction(self):
        self.auction_next_fish()

    def auction_next_fish(self):
        # To be implemented in subclasses
        pass

    def send_fish_info(self):
        auction = self.current_auction
        if not auction['sold']:
            self.log_info(
                f"Auctioning Fish {auction['product_number']}: Type {auction['fish_type']}, Price {auction['current_price']}."
            )
            product_info = {
                'message_type': 'auction_info',
                "product_number": auction['product_number'],
                "product_type": auction['fish_type'],
                "price": auction['current_price']
            }
            self.send('publish_channel', product_info)
            self.timer = self.after(1, self.check_for_replies, alias='price_decrement_timer')

    def on_bid(self, bid):
        self.log_info(f"Received bid: {bid}")
        merchant_id = bid.get('merchant_id')
        auction = self.current_auction
        if auction and not auction['sold']:
            if bid.get('product_number') == auction['product_number']:
                # Sell the fish
                self.log_info(f"Fish {auction['product_number']} sold to Merchant {merchant_id} at price {auction['current_price']}.")
                auction['sold'] = True
                self.transactions.append({
                    'Product': auction['product_number'],
                    'SellPrice': auction['current_price'],
                    'Merchant': merchant_id
                })
                # Stop the timer
                self.stop_timer('price_decrement_timer')
                # Send confirmation
                confirmation = {
                    'message_type': 'confirmation',
                    'status': 'confirmed',
                    'product_number': auction['product_number'],
                    'merchant_id': merchant_id,
                    'price': auction['current_price'],
                    'product_type': auction['fish_type']
                }
                self.send('publish_channel', confirmation)
                # Move to the next auction
                self.auction_next_fish()
        # If fish already sold or bid is invalid, ignore

    def check_for_replies(self, *args, **kwargs):
        # To be implemented in subclasses
        pass

    def on_stop(self):
        log_transactions(self.transactions)


class OperatorInfinite(Operator):
    def on_init(self):
        super().on_init()
        self.fish_in_stock = 30
        self.unsold_count = 0
        self.max_unsold = 3

    def auction_next_fish(self):
        if self.fish_in_stock > 0 and self.unsold_count < self.max_unsold:
            fish_type = self.fish_types[self.fish_index % len(self.fish_types)]
            self.fish_index += 1
            self.fish_in_stock -= 1
            self.current_auction = {
                'fish_type': fish_type,
                'product_number': self.fish_index,
                'current_price': 30,
                'bottom_price': 10,
                'price_decrement': 2,
                'sold': False
            }
            self.send_fish_info()
        else:
            self.log_info("Auction ended.")
            self.running = False  # Set running to False when auction ends

    def check_for_replies(self, *args, **kwargs):
        auction = self.current_auction
        if not auction['sold']:
            auction['current_price'] -= auction['price_decrement']
            if auction['current_price'] >= auction['bottom_price']:
                self.send_fish_info()
            else:
                self.log_info(f"Fish {auction['product_number']} was not sold.")
                self.unsold_count += 1
                self.transactions.append({
                    'Product': auction['product_number'],
                    'SellPrice': 0,
                    'Merchant': 0  # Indicate unsold
                })
                self.auction_next_fish()


class OperatorFinite(Operator):
    def on_init(self):
        super().on_init()
        self.total_fish_to_sell = self.get_attr('total_fish_to_sell')
        self.fish_sold_count = 0

    def auction_next_fish(self):
        if self.fish_sold_count < self.total_fish_to_sell:
            fish_type = self.fish_types[self.fish_index % len(self.fish_types)]
            self.fish_index += 1
            self.current_auction = {
                'fish_type': fish_type,
                'product_number': self.fish_index,
                'current_price': 30,
                'bottom_price': 10,
                'price_decrement': 2,
                'sold': False
            }
            self.send_fish_info()
        else:
            self.log_info("Auction ended after selling the specified number of fish.")
            self.running = False  # Set running to False when auction ends

    def check_for_replies(self, *args, **kwargs):
        auction = self.current_auction
        if not auction['sold']:
            auction['current_price'] -= auction['price_decrement']
            if auction['current_price'] >= auction['bottom_price']:
                self.send_fish_info()
            else:
                self.log_info(f"Fish {auction['product_number']} was not sold.")
                self.transactions.append({
                    'Product': auction['product_number'],
                    'SellPrice': 0,
                    'Merchant': 0  # Indicate unsold
                })
                self.fish_sold_count += 1  # Increment the sold count for unsold fish
                self.auction_next_fish()


    def on_bid(self, bid):
        self.log_info(f"Received bid: {bid}")
        merchant_id = bid.get('merchant_id')
        auction = self.current_auction
        if auction and not auction['sold']:
            if bid.get('product_number') == auction['product_number']:
                # Sell the fish
                self.log_info(
                    f"Fish {auction['product_number']} sold to Merchant {merchant_id} at price {auction['current_price']}."
                )
                auction['sold'] = True
                self.transactions.append({
                    'Product': auction['product_number'],
                    'SellPrice': auction['current_price'],
                    'Merchant': merchant_id
                })
                # Increment fish_sold_count for sold fish
                self.fish_sold_count += 1

                # Stop the timer
                self.stop_timer('price_decrement_timer')

                # Send confirmation with quality
                confirmation = {
                    'message_type': 'confirmation',
                    'status': 'confirmed',
                    'product_number': auction['product_number'],
                    'merchant_id': merchant_id,
                    'price': auction['current_price'],
                    'product_type': auction['fish_type'],
                    'quality': auction.get('quality')  # Include quality in confirmation
                }
                self.send('publish_channel', confirmation)

                # Move to the next auction
                self.auction_next_fish()


class OperatorInfiniteQuality(OperatorInfinite):
    def auction_next_fish(self):
        if self.fish_in_stock > 0 and self.unsold_count < self.max_unsold:
            fish_type = self.fish_types[self.fish_index % len(self.fish_types)]
            fish_quality = random.choice(['good', 'normal', 'bad'])
            self.fish_index += 1
            self.fish_in_stock -= 1
            self.current_auction = {
                'fish_type': fish_type,
                'quality': fish_quality,
                'product_number': self.fish_index,
                'current_price': 30,
                'bottom_price': 10,
                'price_decrement': 2,
                'sold': False
            }
            self.send_fish_info()
        else:
            self.log_info("Auction ended.")
            self.running = False  # Set running to False when auction ends

    def send_fish_info(self):
        auction = self.current_auction
        if not auction['sold']:
            self.log_info(
                f"Auctioning Fish {auction['product_number']}: Type {auction['fish_type']}, "
                f"Quality {auction['quality']}, Price {auction['current_price']}."
            )
            product_info = {
                'message_type': 'auction_info',
                'product_number': auction['product_number'],
                'product_type': auction['fish_type'],
                'quality': auction['quality'],
                'price': auction['current_price']
            }
            self.send('publish_channel', product_info)
            self.timer = self.after(1, self.check_for_replies, alias='price_decrement_timer')


# New OperatorFiniteQuality subclass with quality
class OperatorFiniteQuality(OperatorFinite):
    def auction_next_fish(self):
        if self.fish_sold_count < self.total_fish_to_sell:
            fish_type = self.fish_types[self.fish_index % len(self.fish_types)]
            fish_quality = random.choice(['good', 'normal', 'bad'])
            self.fish_index += 1
            self.current_auction = {
                'fish_type': fish_type,
                'quality': fish_quality,
                'product_number': self.fish_index,
                'current_price': 30,
                'bottom_price': 10,
                'price_decrement': 2,
                'sold': False
            }
            self.send_fish_info()
        else:
            self.log_info("Auction ended after selling the specified number of fish.")
            self.running = False  # Set running to False when auction ends

    def send_fish_info(self):
        auction = self.current_auction
        if not auction['sold']:
            self.log_info(
                f"Auctioning Fish {auction['product_number']}: Type {auction['fish_type']}, "
                f"Quality {auction['quality']}, Price {auction['current_price']}."
            )
            product_info = {
                'message_type': 'auction_info',
                'product_number': auction['product_number'],
                'product_type': auction['fish_type'],
                'quality': auction['quality'],
                'price': auction['current_price']
            }
            self.send('publish_channel', product_info)
            self.timer = self.after(1, self.check_for_replies, alias='price_decrement_timer') 

### Simulacion de la Subasta del Peix

El siguiente código representa el **programa principal** de la simulación de subasta del Peix.
A continuación, se explica cómo interactúan los agentes en la simulación:

---

### **Propósito del Código**
1. **Configuración de Agentes:**
   - Lee un archivo de configuración para inicializar el **operador** (único propietario de la subasta) y los **comerciantes** (posibles compradores).
   - Crea y conecta agentes al sistema de comunicación, estableciendo roles y preferencias.

2. **Inicio de la Subasta:**
   - El operador publica información sobre cada pescado y reduce gradualmente los precios en cada ronda.
   - Los comerciantes reciben mensajes del operador, evalúan si realizar una oferta y envían sus pujas al operador.

3. **Registro de Resultados:**
   - Se registran los detalles del inventario de los comerciantes y las transacciones realizadas durante la subasta.

---

### **Resumen del flujo de trabajo de la Subasta**
1. **Creación de Agentes:**
   - **Operador:** Se inicializa un único operador basado en el tipo definido en la configuración (infinito, finito, con o sin calidad).
   - **Comerciantes:** Se crean comerciantes básicos, ricos y pobres según el número especificado en la configuración, asignándoles un presupuesto y preferencias aleatorias.

2. **Conexión de Agentes:**
   - Los comerciantes se conectan al canal de publicación del operador para recibir información sobre la subasta.
   - Los comerciantes también establecen un canal de envío para enviar sus pujas al operador.

3. **Interacción de Agentes:**
   - **Operador:** Publica productos y procesa las pujas recibidas, determinando si un pescado se vende o se descarta.
   - **Comerciantes:** Deciden si comprar un pescado basándose en su presupuesto, preferencias y el precio actual.

4. **Finalización:**
   - Una vez que la subasta termina, el inventario final de los comerciantes y las transacciones se registran en archivos.


In [ ]:
from osbrain import run_nameserver, run_agent, Agent
import csv
import random
from datetime import datetime
import time
import logging
from threading import Thread
from merchants import BasicMerchant, RichMerchant, PoorMerchant
from operators import OperatorInfinite, OperatorFinite, OperatorInfiniteQuality, OperatorFiniteQuality



# Set logging level to DEBUG for osBrain
logging.getLogger('osbrain').setLevel(logging.DEBUG)



def read_config_file(file_path):
    """
    Reads and parses the configuration file.
    """
    config = {}
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line and not line.startswith("#"):  # Ignore comments and empty lines
                    key, value = line.split(":")
                    config[key.strip()] = value.strip()
    except Exception as e:
        print(f"Error reading configuration file: {e}")
        exit(1)
    return config


def log_merchants_inventory(merchants):
    """
    Logs each merchant's inventory details to a plain text file.
    """
    date_str = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'merchant_inventory_{date_str}.txt'
    
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("=== Merchant Inventory Report ===\n\n")
        for merchant in merchants:
            merchant_name = merchant.get_name()  # Use the exposed method
            merchant_budget = merchant.get_attr('budget')
            inventory = merchant.get_attr('inventory')

            # Write Merchant Header
            file.write(f"Merchant: {merchant_name}\n")
            file.write(f"Remaining Budget: {merchant_budget}\n")
            file.write("Inventory:\n")
            
            # Check if inventory is empty
            if not inventory:
                file.write("  - No items in inventory\n")
            else:
                # Write each fish in the inventory
                for product_number, details in inventory.items():
                    fish_type = details.get('type', 'Unknown')
                    quality = details.get('quality', 'N/A')
                    price = details.get('price', 'N/A')
                    file.write(f"  - Product {product_number}: Type {fish_type}, Quality {quality}, Price {price}\n")
            
            file.write("\n")  # Add spacing between merchants
        file.write("=== End of Report ===\n")
    
    print(f"Merchant inventory report saved to '{filename}'.")







def log_transactions(transactions):
    date_str = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    with open(f'log_{date_str}.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Product', 'SellPrice', 'Merchant'])
        writer.writeheader()
        for transaction in transactions:
            writer.writerow(transaction)


def log_setup(merchants_info):
    date_str = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    with open(f'setup_{date_str}.csv', mode='w', newline='', encoding='utf-8') as file:
        # Include 'Type' in the fieldnames
        writer = csv.DictWriter(file, fieldnames=['Merchant', 'Type', 'Preference', 'Budget'])
        writer.writeheader()
        for info in merchants_info:
            writer.writerow(info)




    # Main program execution


if __name__ == '__main__':
    ns = run_nameserver()

    # Read configuration file
    config_file = "config.txt"
    config = read_config_file(config_file)

    # Extract inputs
    operator_type = int(config.get('operator_type', 1))
    total_fish_to_sell = int(config.get('total_fish_to_sell', 10))
    num_basic_merchants = int(config.get('num_basic_merchants', 0))
    num_rich_merchants = int(config.get('num_rich_merchants', 0))
    num_poor_merchants = int(config.get('num_poor_merchants', 0))

    operator = None
    use_quality = False

    # Initialize the operator based on configuration
    if operator_type == 1:
        operator = run_agent('OperatorInfinite', base=OperatorInfinite)
    elif operator_type == 2:
        operator = run_agent(
            'OperatorFinite',
            base=OperatorFinite,
            attributes={'total_fish_to_sell': total_fish_to_sell}
        )
    elif operator_type == 3:
        operator = run_agent('OperatorInfiniteQuality', base=OperatorInfiniteQuality)
        use_quality = True
    elif operator_type == 4:
        operator = run_agent(
            'OperatorFiniteQuality',
            base=OperatorFiniteQuality,
            attributes={'total_fish_to_sell': total_fish_to_sell}
        )
        use_quality = True
    else:
        print("Invalid operator type in configuration file.")
        ns.shutdown()
        exit()

    print("Quality logic is enabled for merchants.") if use_quality else None

    # Create merchants
    merchants = []  # List to hold all merchant agents
    merchants_info = []  # List to log merchant details

    publish_address = operator.addr('publish_channel')
    bid_address = operator.addr('bid_channel')

    def create_merchants(num_merchants, merchant_class, budget):
        """Creates a specified number of merchants and connects them to the operator."""
        for i in range(1, num_merchants + 1):
            merchant_name = f'{merchant_class.__name__}_{i}'
            merchant = run_agent(merchant_name, base=merchant_class)
            merchant.set_attr(budget=budget)
            merchant.connect(publish_address, handler='on_operator_message')
            merchant.bind('PUSH', alias='bid_channel')
            merchant.connect(bid_address, alias='bid_channel')
            merchants.append(merchant)
            merchants_info.append({
                'Merchant': merchant_name,
                'Type': merchant_class.__name__,
                'Preference': merchant.get_attr('preference'),
                'Budget': merchant.get_attr('budget')
            })

    # Use inputs from the config file
    create_merchants(num_basic_merchants, BasicMerchant, 100)
    create_merchants(num_rich_merchants, RichMerchant, 500)
    create_merchants(num_poor_merchants, PoorMerchant, 50)

    # Log setup and start auction
    log_setup(merchants_info)
    operator.start_auction()

    # Wait for the auction to finish
    while operator.get_attr('running'):
        time.sleep(1)

    log_merchants_inventory(merchants)

    # Shutdown all agents
    operator.shutdown()
    for merchant in merchants:
        merchant.shutdown()
    ns.shutdown()